In [2]:
import pandas as pd
import PyPDF2

In [102]:
def extract_text_from_pdf(pdf_path: str) -> list[tuple[int, str]]:
    """Extract text from a PDF file.

    Args:
        pdf_path (str): Path to the PDF file.

    Returns:
        list[tuple[int, str]]: List of tuples where the first element is the page number and the second element is the text on that page.
    """
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        return {f"Page {i}": page.extract_text() for i, page in enumerate(reader.pages)}


def get_month(text: str) -> str:
    """Get the month from the first page of the PDF.

    Args:
        first_page (str): Text from the first page of the PDF.

    Returns:
        str: The month in the format "January", "February", etc.
    """
    month_names = {
        "01": "January",
        "02": "February",
        "03": "March",
        "04": "April",
        "05": "May",
        "06": "June",
        "07": "July",
        "08": "August",
        "09": "September",
        "10": "October",
        "11": "November",
        "12": "December",
    }
    date_range = text.split("\n")[2]
    month = date_range.split("-")[1]
    return month_names[month]


def clean_sales_data(text: str) -> str:
    """Clean the sales data from the PDF.

    Args:
        text (str): Text from the second page of the PDF.

    Returns:
        str: Cleaned sales data.
    """
    text = text.replace("Försäljning efter produkt\n", "")
    text = text.replace(
        "Namn Sålt Returnerat Exklusive moms Inklusive moms",
        "Namn Sålt Returnerat Exklusive-moms Inklusive-moms",
    )
    text = text.replace("PANINI, ", "PANINI,")
    text = text.replace("KAFFE, ", "KAFFE,")
    text = text.replace("TRIUMF, ", "TRIUMF,")
    text = text.replace("SÖTA, ", "SÖTA,")
    text = text.replace("\xa0", "")
    rows = text.split("\n")
    sales_data = [row.split(" ") for row in rows]
    sales_data[0].remove("Returnerat")
    return sales_data


def join_product_names(sales_data: list[list[str]]) -> list[list[str]]:
    """Join product names that have been split into multiple elements.

    Args:
        sales_data (list[list[str]]): Sales data.

    Returns:
        list[list[str]]: Sales data with product names joined.
    """
    processed_data = [sales_data[0]]
    for item in sales_data[1:]:
        new_item = []
        name = ""
        i = 0
        while i < len(item):
            if item[i].isdigit():
                break
            name += item[i] + " "
            i += 1
        new_item.append(name.strip())
        new_item.extend(item[i:])
        processed_data.append(new_item)
    return processed_data[:-3]


def convert_to_dataframe(sales_data: list[list[str]], month: str) -> pd.DataFrame:
    """Convert sales data to a DataFrame.

    Args:
        sales_data (list[list[str]]): Sales data.
        month (str): The month in the format "January", "February", etc.

    Returns:
        pd.DataFrame: Sales data as a DataFrame.
    """
    df = pd.DataFrame(sales_data[1:], columns=sales_data[0])
    df["Month"] = month
    return df

In [103]:
pdf  = extract_text_from_pdf('Zettle-Sales\Zettle-Sales-Report-20220701-20220731.pdf')
pdf 


<>:1: SyntaxWarning: invalid escape sequence '\Z'
<>:1: SyntaxWarning: invalid escape sequence '\Z'
C:\Users\victo\AppData\Local\Temp\ipykernel_19972\1835976533.py:1: SyntaxWarning: invalid escape sequence '\Z'
  pdf  = extract_text_from_pdf('Zettle-Sales\Zettle-Sales-Report-20220701-20220731.pdf')


{'Page 0': 'Dahn Wester AB\nOrganisationsnummer: 5593748360Försäljningsrapport\n2022-07-01 - 2022-07-31\nFörsäljningsöversikt\nExklusive moms Moms Inklusive moms\nFörsäljning 181\xa0193,35\xa0kr 21\xa0764,65\xa0kr 202\xa0958,00\xa0kr\nTotalt 181\xa0193,35\xa0kr 21\xa0764,65\xa0kr 202\xa0958,00\xa0kr\nFörsäljning per säljkanal\nFörsäljning Antal Exklusive moms Inklusive moms\nKassasystem 1455 181\xa0193,35\xa0kr 202\xa0958,00\xa0kr\nTotalt 1455 181\xa0193,35\xa0kr 202\xa0958,00\xa0kr\nFörsäljning per momssats\nExklusive moms Moms Inklusive moms\n12\xa0% Moms 181\xa0033,35\xa0kr 21\xa0724,65\xa0kr 202\xa0758,00\xa0kr\n25\xa0% Moms 160,00\xa0kr 40,00\xa0kr 200,00\xa0kr\nTotalt 181\xa0193,35\xa0kr 21\xa0764,65\xa0kr 202\xa0958,00\xa0kr\nBetalningar och avgifter med Zettle\nBetalningar och avgifter insamlade med Zettle Belopp Avgift\nKort via kortläsare 185\xa0591,00\xa0kr -3\xa0435,33\xa0kr\nSumma av Zettle-betalningar 185\xa0591,00\xa0kr\nSumma avgifter -3\xa0435,33\xa0kr\nSumma efter avg

In [104]:
first_page = pdf['Page 0']
rows = first_page.split("\n")
rows[29] = "Namn Sålt Returnerat Exklusive-moms Inklusive-moms"
sales_data = [row.split(" ") for row in rows]

In [105]:
rows[29]

'Namn Sålt Returnerat Exklusive-moms Inklusive-moms'

In [106]:
sales_data[29:]

[['Namn', 'Sålt', 'Returnerat', 'Exklusive-moms', 'Inklusive-moms'],
 ['Läsk', '675', '18\xa0081,32\xa0kr', '20\xa0250,00\xa0kr'],
 ['PANINI,',
  'Italian',
  'Stallion',
  '414',
  '31\xa0419,42\xa0kr',
  '35\xa0190,00\xa0kr'],
 ['KAFFE,',
  'Espresso',
  'm.',
  'Mjölk',
  '280',
  '7\xa0500,65\xa0kr',
  '8\xa0400,00\xa0kr'],
 ['KAFFE,', 'Bryggkaffe', '238', '5\xa0312,17\xa0kr', '5\xa0950,00\xa0kr'],
 ['Strips', '237', '10\xa0580,16\xa0kr', '11\xa0850,00\xa0kr'],
 ['PANINI,',
  'Peppes',
  'Pere',
  '235',
  '17\xa0834,51\xa0kr',
  '19\xa0975,00\xa0kr'],
 ['PANINI,',
  'Calles',
  'Cubano',
  '225',
  '17\xa0075,55\xa0kr',
  '19\xa0125,00\xa0kr']]

In [107]:
sales_data = sales_data[29:]

In [108]:
for row in sales_data:
    for index, element in enumerate(row):
        row[index] = element.replace("Försäljning efter produkt\n", "")
        row[index] = element.replace(
            "Namn Sålt Returnerat Exklusive moms Inklusive moms",
            "Namn Sålt Returnerat Exklusive-moms Inklusive-moms",
        )
        row[index] = element.replace("PANINI, ", "PANINI,")
        row[index] = element.replace("KAFFE, ", "KAFFE,")
        row[index] = element.replace("TRIUMF, ", "TRIUMF,")
        row[index] = element.replace("SÖTA, ", "SÖTA,")
        row[index] = element.replace("\xa0", "")
sales_data

[['Namn', 'Sålt', 'Returnerat', 'Exklusive-moms', 'Inklusive-moms'],
 ['Läsk', '675', '18081,32kr', '20250,00kr'],
 ['PANINI,', 'Italian', 'Stallion', '414', '31419,42kr', '35190,00kr'],
 ['KAFFE,', 'Espresso', 'm.', 'Mjölk', '280', '7500,65kr', '8400,00kr'],
 ['KAFFE,', 'Bryggkaffe', '238', '5312,17kr', '5950,00kr'],
 ['Strips', '237', '10580,16kr', '11850,00kr'],
 ['PANINI,', 'Peppes', 'Pere', '235', '17834,51kr', '19975,00kr'],
 ['PANINI,', 'Calles', 'Cubano', '225', '17075,55kr', '19125,00kr']]

In [109]:
second_page = pdf['Page 1']
second_page = second_page.replace("PANINI, ", "PANINI,")
second_page = second_page.replace("KAFFE, ", "KAFFE,")
second_page = second_page.replace("TRIUMF, ", "TRIUMF,")
second_page = second_page.replace("SÖTA, ", "SÖTA,")
second_page = second_page.replace("\xa0", "")
rows = second_page.split("\n")
sales_data_2 = [row.split(" ") for row in rows]
sales_data_2

[['Citron', '204', '4553,31kr', '5100,00kr'],
 ['GUTE', '150', '4687,50kr', '5250,00kr'],
 ['PANINI,Jacks', 'Bagnat', '148', '11231,92kr', '12580,00kr'],
 ['ÖL', '139', '4964,04kr', '5560,00kr'],
 ['Lime', '124', '2767,71kr', '3100,00kr'],
 ['PANINI,Abbes', 'Aubergine', '113', '8575,68kr', '9605,00kr'],
 ['Kids', 'Croque', '112', '4500,15kr', '5040,00kr'],
 ['KAFFE,Dubbel', 'Espresso', '106', '2365,92kr', '2650,00kr'],
 ['KAFFE,Espresso', '92', '1232,03kr', '1380,00kr'],
 ['SÖTA,Cannolo', 'Siciliano', '66', '1768,04kr', '1980,00kr'],
 ['TRIUMF,Skruven', '66', '1001,87kr', '1122,00kr'],
 ['TRIUMF,Snickers', '58', '1242,94kr', '1392,00kr'],
 ['TRIUMF,Dumle', '54', '1350,00kr', '1512,00kr'],
 ['SÖTA,Apolline', '50', '1116,00kr', '1250,00kr'],
 ['TRIUMF,Brynt', 'Smör', '49', '1225,00kr', '1372,00kr'],
 ['Melonsallad', '46', '1232,29kr', '1380,00kr'],
 ['TRIUMF,Sandwich', '42', '750,05kr', '840,00kr'],
 ['TRIUMF,Salt', 'Lakrits', '41', '1025,00kr', '1148,00kr'],
 ['P-PANINI,Italian', 'Stall

In [110]:
sales_data.extend(sales_data_2)
sales_data

[['Namn', 'Sålt', 'Returnerat', 'Exklusive-moms', 'Inklusive-moms'],
 ['Läsk', '675', '18081,32kr', '20250,00kr'],
 ['PANINI,', 'Italian', 'Stallion', '414', '31419,42kr', '35190,00kr'],
 ['KAFFE,', 'Espresso', 'm.', 'Mjölk', '280', '7500,65kr', '8400,00kr'],
 ['KAFFE,', 'Bryggkaffe', '238', '5312,17kr', '5950,00kr'],
 ['Strips', '237', '10580,16kr', '11850,00kr'],
 ['PANINI,', 'Peppes', 'Pere', '235', '17834,51kr', '19975,00kr'],
 ['PANINI,', 'Calles', 'Cubano', '225', '17075,55kr', '19125,00kr'],
 ['Citron', '204', '4553,31kr', '5100,00kr'],
 ['GUTE', '150', '4687,50kr', '5250,00kr'],
 ['PANINI,Jacks', 'Bagnat', '148', '11231,92kr', '12580,00kr'],
 ['ÖL', '139', '4964,04kr', '5560,00kr'],
 ['Lime', '124', '2767,71kr', '3100,00kr'],
 ['PANINI,Abbes', 'Aubergine', '113', '8575,68kr', '9605,00kr'],
 ['Kids', 'Croque', '112', '4500,15kr', '5040,00kr'],
 ['KAFFE,Dubbel', 'Espresso', '106', '2365,92kr', '2650,00kr'],
 ['KAFFE,Espresso', '92', '1232,03kr', '1380,00kr'],
 ['SÖTA,Cannolo', 'S

In [112]:
sales_data = join_product_names(sales_data)
sales_data

[['Namn', 'Sålt', 'Returnerat', 'Exklusive-moms', 'Inklusive-moms'],
 ['Läsk', '675', '18081,32kr', '20250,00kr'],
 ['PANINI, Italian Stallion', '414', '31419,42kr', '35190,00kr'],
 ['KAFFE, Espresso m. Mjölk', '280', '7500,65kr', '8400,00kr'],
 ['KAFFE, Bryggkaffe', '238', '5312,17kr', '5950,00kr'],
 ['Strips', '237', '10580,16kr', '11850,00kr'],
 ['PANINI, Peppes Pere', '235', '17834,51kr', '19975,00kr'],
 ['PANINI, Calles Cubano', '225', '17075,55kr', '19125,00kr'],
 ['Citron', '204', '4553,31kr', '5100,00kr'],
 ['GUTE', '150', '4687,50kr', '5250,00kr'],
 ['PANINI,Jacks Bagnat', '148', '11231,92kr', '12580,00kr'],
 ['ÖL', '139', '4964,04kr', '5560,00kr'],
 ['Lime', '124', '2767,71kr', '3100,00kr'],
 ['PANINI,Abbes Aubergine', '113', '8575,68kr', '9605,00kr'],
 ['Kids Croque', '112', '4500,15kr', '5040,00kr'],
 ['KAFFE,Dubbel Espresso', '106', '2365,92kr', '2650,00kr'],
 ['KAFFE,Espresso', '92', '1232,03kr', '1380,00kr'],
 ['SÖTA,Cannolo Siciliano', '66', '1768,04kr', '1980,00kr'],
 

In [119]:
sales_data[0].remove("Returnerat")

In [120]:
df = convert_to_dataframe(sales_data, get_month(first_page))

In [121]:
df.to_csv("processed_sales_data/prego_sales_july.csv", index=False)

In [122]:
pdf_file.close()

In [3]:
prego_sales = pd.read_csv("processed_sales_data/prego_sales_june_july_august.csv")
prego_sales.head()

,Namn,Sålt,Exklusive-moms,Inklusive-moms,Month
0,Läsk,75,"2009,02kr","2250,00kr",June
1,"PANINI,Italian Stallion",42,"3187,47kr","3570,00kr",June
2,"KAFFE,Bryggkaffe",40,"892,80kr","1000,00kr",June
3,GUTE,34,"1062,50kr","1190,00kr",June
4,"PANINI,Peppes Pere",29,"2200,84kr","2465,00kr",June


In [9]:
paninis = prego_sales["Namn"].str.contains("PANINI")
paninis = prego_sales[paninis]
paninis.Namn.unique()

array(['PANINI,Italian Stallion', 'PANINI,Peppes Pere',
       'P-PANINI,Stallion', 'PANINI,Calles Cubano',
       'PANINI,Arons Aubergine', 'P-PANINI,Calles', 'PANINI,Jacks Bagnat',
       'P-PANINI,Peppes', 'P-PANINI,Aubergine', 'P-PANINI,Jacks',
       'P-PANINI,Stoffe', 'PANINI, Italian Stallion',
       'PANINI, Peppes Pere', 'PANINI, Calles Cubano',
       'PANINI,Abbes Aubergine', 'P-PANINI,Italian Stallion',
       'P-PANINI,Jacks Bagnat', 'P-PANINI,Peppes Pere',
       'P-PANINI,Calles Cubano', 'P-PANINI,Abbes Aubergine'], dtype=object)

In [10]:
panini_name_converter = {
    lambda name: name.__contains__("Stallion"): "Panini Italian Stallion",
}

for name in paninis.Namn.unique():
    for key, value in panini_name_converter.items():
        if key(name):
            paninis.Namn = paninis.Namn.replace(name, value)
paninis.Namn.unique()

C:\Users\victo\AppData\Local\Temp\ipykernel_30976\2010945350.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  paninis.Namn = paninis.Namn.replace(name, value)


array(['Italian Stallion', 'PANINI,Peppes Pere', 'PANINI,Calles Cubano',
       'PANINI,Arons Aubergine', 'P-PANINI,Calles', 'PANINI,Jacks Bagnat',
       'P-PANINI,Peppes', 'P-PANINI,Aubergine', 'P-PANINI,Jacks',
       'P-PANINI,Stoffe', 'PANINI, Peppes Pere', 'PANINI, Calles Cubano',
       'PANINI,Abbes Aubergine', 'P-PANINI,Jacks Bagnat',
       'P-PANINI,Peppes Pere', 'P-PANINI,Calles Cubano',
       'P-PANINI,Abbes Aubergine'], dtype=object)